# MSV / SS 2023 - Übung 7 - Lösung der Hausaufgaben

## Hausaufgaben

### Übung 7.1

Annotieren Sie die Daten im 'reviews_toannotate.csv' (mit "pos" oder "neg" in der in der letzten Spalte (csv mit ";" als delimiter) und und berechnen Sie die Agreement (kappa und alpha) mit der bereits kommentierten Datei 'reviews_gold.csv'. 

In [2]:
import pandas as pd
from nltk.metrics.agreement import AnnotationTask

In [3]:
annotator1_data = pd.read_csv('reviews_gold.csv',delimiter=';', header=None)
print(annotator1_data)

                                                    0    1
0   an entertaining 2 hours awaits the audience in...  pos
1   suddenly warren beatty doesn't look so absurd ) .  neg
2     by the time they show you , it doesn't matter .  neg
3   deniro is also affable , tough on the outside ...  pos
4   its 100-minute running time feels like 100 yea...  neg
5   this film would definitely not have been as go...  pos
6   because of freeman's powerful presence , he ca...  pos
7   every shot is choreographed as a portrait -- a...  pos
8   in short , rudolph has created a world that it...  pos
9   kick boxing deaths and conflicted nuns are not...  neg
10  the costumes are remarkable and have jean-paul...  pos
11  i was surprised how expressive an actor he pro...  pos
12  and if i spoiled it for you , good , i saved y...  neg
13  i like his quick-edit style , because it's abr...  pos
14  the problems in logic are flaws , but don't ru...  pos
15  connery plays his character with the exact amo...  p

In [4]:
annotator2_data = pd.read_csv('reviews_annotated.csv',delimiter=';', header=None)
print(annotator2_data)

                                                    0    1
0   an entertaining 2 hours awaits the audience in...  pos
1   suddenly warren beatty doesn't look so absurd ) .  neg
2     by the time they show you , it doesn't matter .  neg
3   deniro is also affable , tough on the outside ...  pos
4   its 100-minute running time feels like 100 yea...  neg
5   this film would definitely not have been as go...  neg
6   because of freeman's powerful presence , he ca...  neg
7   every shot is choreographed as a portrait -- a...  pos
8   in short , rudolph has created a world that it...  pos
9   kick boxing deaths and conflicted nuns are not...  neg
10  the costumes are remarkable and have jean-paul...  pos
11  i was surprised how expressive an actor he pro...  pos
12  and if i spoiled it for you , good , i saved y...  neg
13  i like his quick-edit style , because it's abr...  pos
14  the problems in logic are flaws , but don't ru...  neg
15  connery plays his character with the exact amo...  p

In [5]:
matrix = [annotator1_data[1],
         annotator2_data[1]] # replace with your data
header = pd.Series(range(1,40))
arrays = ['gold', 'you']
annotation = pd.DataFrame(matrix, index=arrays, columns=header,)
annotation

,1,2,3,4,5,6,7,8,9,10,...,30,31,32,33,34,35,36,37,38,39
gold,neg,neg,pos,neg,pos,pos,pos,pos,neg,pos,...,neg,pos,neg,pos,neg,neg,neg,neg,pos,neg
you,neg,neg,pos,neg,neg,neg,pos,pos,neg,pos,...,neg,pos,neg,pos,neg,neg,neg,neg,pos,neg


In [6]:
annotation_tuples = annotation.reset_index().melt(id_vars=['index'])
annotation_tuples.columns = ["coder", "item", "code"]
annotation_tuples = list(zip(annotation_tuples.coder, annotation_tuples.item, annotation_tuples.code))
annotation_tuplestask = AnnotationTask(data=annotation_tuples)

In [7]:
annotation_tuplestask.avg_Ao()

0.8717948717948718

In [8]:
annotation_tuplestask.kappa()

0.7483870967741936

In [9]:
annotation_tuplestask.alpha()

0.7482014388489209

### Übung 7.2

Finden Sie die Entitäten in diesem Text mit Hilfe des Spacy-NER-Modus. Suchen Sie nach möglichen Fehlern und überlegen Sie, wie Sie diese beheben können.

In [12]:
harrypotter_file = 'harrypotter.txt'
harrypotter_txt = open(harrypotter_file).read()
# replace \n with whitespace
harrypotter_txt = harrypotter_txt.replace('\n',' ')
harrypotter_txt

'"Nobody was very attentive in lessons, being much more interested in the arrival that evening of the people from Beauxbatons and Durmstrang; even Potions was more bearable than usual, as it was half an hour shorter.  When the bell rang early, Harry, Ron, and Hermione hurried up to Gryffindor Tower, deposited their bags and books as they had been instructed, pulled on their cloaks, and rushed back downstairs into the entrance hall. The Heads of Houses were ordering their students into lines.  "Weasley, straighten your hat," Professor McGonagall snapped at Ron.  "Miss Patil, take that ridiculous thing out of your hair."  Parvati scowled and removed a large ornamental butterfly from the end of her plait.  "Follow me, please," said Professor McGonagall.  "First years in front. . . no pushing.. ."  They filed down the steps and lined up in front of the castle. It was a cold, clear evening; dusk was falling and a pale, transparent-looking moon was already shining over the Forbidden Forest. 

In [55]:
import re
import spacy
nlp_en = spacy.load("en_core_web_md") 

In [56]:
doc = nlp_en(harrypotter_txt)

In [57]:
from spacy import displacy
displacy.render(doc, style="ent")

#### "Potions" als Hogwarts Subject und Beauxbatons / Durmstrang als Wizarding Schools

In [101]:
nlp_en = spacy.load("en_core_web_md") 

In [103]:
ner = nlp_en.get_pipe("ner")
ner.add_label("HOGWARTS_SUBJ")
ner.add_label("WIZ_SCHOOLS")

patterns = [{"label": "HOGWARTS_SUBJ", "pattern": "Astronomy"},
            {"label": "HOGWARTS_SUBJ", "pattern": "Charms"},
            {"label": "HOGWARTS_SUBJ", "pattern": "Defence Against the Dark Arts"},
            {"label": "HOGWARTS_SUBJ", "pattern": "Herbology"},
            {"label": "HOGWARTS_SUBJ", "pattern": "Potions"},
            {"label": "WIZ_SCHOOLS", "pattern": "Beauxbatons"},
            {"label": "WIZ_SCHOOLS", "pattern": "Durmstrang"}
           ]
ruler = nlp_en.add_pipe("entity_ruler", before='ner')
ruler.add_patterns(patterns)
doc2 = nlp_en(harrypotter_txt)

In [104]:
displacy.render(doc2, style="ent")